Tipo de variables:

Caracteristicas individuales:
1. Sexo
2. Edad
3. Previsión o seguro de salud

Caracteristicas del centro de salud:
1. Nombre
2. Público o privado

Variables asociadas al disgnostico:
1. Nombre del problema de salud (y código problema de salud)

Variables asociadas al individuo, centro de salud y enfermedad de forma simultanea:
1. Dias de estadia en el centro de salud
2. Condicion de egreso (vivo o muerto)

Preguntas Guia:
1. ¿Cual es la tendencia en la evolución de la prevalencia de los problemas de salud a lo largo de los años? 2001-2024
2. ¿Hay alguna relación entre las caractersiticas individuales (sexo, edad, previsión de salud) y problemas de salud especificos? 2001-2024
3. ¿Que tipo de centros de salud (publicos o privados) y en especifico utilizan más o menos días-cama en la solución de los problemas de salud? 2001-2024 en general y 2001-2020 en especifico
3.1 ¿ En que medida la eficiencia de los centros de salud está ligada a caracteristicas individuales de la población atendida y/o a su financiamiento?
Depende de hallazgos P.2 2001-2024 en general y 2001-2020 en especifico
* Se entiende que un centro de salud es más eficiente si es que logra reducir los días de estada al mínimo posible
y efectivo si es que el paciente egresa vivo. BUSCAR UNA DEFINICION MAS APROPIADA DE EFICIENCIA HOSPITALARIA

# Libraries and Data importation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from unidecode import unidecode

In [3]:
df = pd.read_parquet(r"C:\Users\mirko\Desktop\Curso Data Science DL\ProyectoEDA_Egresos_Hospitalarios_2001-2020\data\processed\Discharges_2001-2024_analysis.parquet")

In [4]:
df.head()

,sex,age,health insurance,healthcare facility type,healthcare facility name,length of stay,discharge condition,primary diagnosis code,primary diagnosis name,year
0,True,"[60, 69]",True,True,Clínica Familia,2,True,C780,TUMOR MALIGNO SECUNDARIO DEL PULMÓN,1970-01-01 00:00:00.000002001
1,True,"[70, 79]",True,True,"Hospital Del Salvador (Santiago, Providencia)",58,True,E145,"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLIC...",1970-01-01 00:00:00.000002001
2,True,"[70, 79]",True,True,"Hospital Del Salvador (Santiago, Providencia)",12,True,J189,"NEUMONIA, NO ESPECIFICADA",1970-01-01 00:00:00.000002001
3,True,"[1, 9]",True,True,Instituto de Neurocirugía Dr. Alfonso Asenjo,1,True,S065,HEMORRAGIA SUBDURAL TRAUMÁTICA,1970-01-01 00:00:00.000002001
4,True,"[80, 120]",True,True,Hospital Dr. Leonardo Guzmán (Antofagasta),7,True,K830,COLANGITIS,1970-01-01 00:00:00.000002001


In [7]:
# Save in an object the 2 columns associated with diagnosis
diagnosis = df[["primary diagnosis code", "primary diagnosis name"]]

In [8]:
# There are not nan codes, so it is possible to assign a name to the nan values in "primary diagnosis name"
diagnosis.isna().sum()

primary diagnosis code          0
primary diagnosis name    4970764
dtype: int64

In [13]:
# Step 2: Filter out rows where the name is None or NaN
diagnosis = diagnosis[diagnosis["primary diagnosis name"].notna()]

In [15]:
# Step 3: Normalize -> lowercase + remove accents
diagnosis["name_clean"] = diagnosis["primary diagnosis name"].map(
    lambda x: unidecode(x).lower()
)

In [16]:
# Step 4: Group by code and take the first normalized name
first_valid = (
    diagnosis.groupby("primary diagnosis code")["name_clean"]
    .first()          # first value after grouping
    .reset_index()
)

# Step 5: Build dictionary
clean_dictionary = dict(
    zip(first_valid["primary diagnosis code"], first_valid["name_clean"])
)

In [17]:
replacement_series = df["primary diagnosis code"].map(clean_dictionary)

In [18]:
df["primary diagnosis name"] = df["primary diagnosis name"].fillna(replacement_series)

In [23]:
df[df["primary diagnosis name"].isna()]["primary diagnosis code"].value_counts()

primary diagnosis code
B04X    407
U109    353
U099    235
A971     41
U129     37
A979     24
A970     19
L987     17
P917      9
e115      5
A972      3
G233      3
z518      3
C61x      2
G835      2
c498      2
A309      2
E502      2
o998      2
A803      1
M493      1
B550      1
L652      1
A798      1
T674      1
B600      1
B050      1
k819      1
A288      1
i660      1
k409      1
A304      1
R452      1
C20x      1
j90x      1
c795      1
Q962      1
d62x      1
j81X      1
k709      1
Q987      1
u071      1
B355      1
R894      1
Name: count, dtype: int64

In [25]:
df[(df["primary diagnosis code"] == "B04X") & (df["primary diagnosis name"].notna())]

,sex,age,health insurance,healthcare facility type,healthcare facility name,length of stay,discharge condition,primary diagnosis code,primary diagnosis name,year


In [27]:
condition = (df["primary diagnosis code"] == "B04X") & (df["primary diagnosis name"].isna())
df.loc[condition, "primary diagnosis name"] = "viruela de los monos"

In [29]:
df = df[df["primary diagnosis name"].notna()]

In [39]:
df["primary diagnosis name"] = df["primary diagnosis name"].apply(lambda x : x.lower())

In [45]:
# Group by code and get the unique names per code
conflict_check = (
    df.groupby("primary diagnosis code")["primary diagnosis name"]
    .unique()
    .reset_index()
)

# Keep only codes with more than one unique name
conflict_check = conflict_check[
    conflict_check["primary diagnosis name"].apply(lambda x: len(x) > 1)
]

# Show the result
print(f"Number of codes with multiple names: {len(conflict_check)}")
print(conflict_check)

conflict_check.to_excel("final_conflict_check.xlsx")

Number of codes with multiple names: 1999
     primary diagnosis code                             primary diagnosis name
0                      A000  [cólera debido a vibrio cholerae o1, biotipo c...
1                      A009  [cólera, no especificado, colera, no especific...
11                     A029  [infección debida a salmonella, no especificad...
18                     A040  [infeccion debida a escherichia coli enteropat...
28                     A050  [intoxicación alimentaria estafilocócica, into...
...                     ...                                                ...
8405                   Z965  [presencia de implante de raíz de diente y de ...
8406                   Z966  [presencia de implante ortopédico articular, p...
8415                   Z980  [estado de derivación intestinal o anastomosis...
8421                   Z992  [dependencia de diálisis renal, dependencia de...
8425                   Z999  [dependencia de máquina y dispositivo capacita...

[1999 row

In [7]:
df["Sex"].value_counts()

Sex
Mujer     14975668
Hombre    10847713
Name: count, dtype: int64

In [28]:
df.shape

(25823381, 10)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25823381 entries, 0 to 25823380
Data columns (total 10 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   Sex                       object  
 1   Age                       int32   
 2   Health insurance          category
 3   Healthcare facility type  category
 4   Healthcare facility name  object  
 5   Length of stay            int32   
 6   Discharge condition       category
 7   Primary diagnosis code    object  
 8   Primary diagnosis name    object  
 9   Year                      int64   
dtypes: category(3), int32(2), int64(1), object(4)
memory usage: 1.2+ GB


In [32]:
df.describe()

,Age,Length of stay,Year
count,2.582338e+07,2.582338e+07,2.582338e+07
mean,3.929132e+01,6.026175e+00,2.010925e+03
std,2.494052e+01,5.566398e+01,5.287845e+00
min,0.000000e+00,1.000000e+00,2.001000e+03
25%,2.000000e+01,1.000000e+00,2.007000e+03
50%,3.600000e+01,3.000000e+00,2.011000e+03
75%,6.000000e+01,6.000000e+00,2.015000e+03
max,1.210000e+02,3.245200e+04,2.020000e+03


# Univariate analysis

In [ ]:
Caracteristicas individuales:
1. Sexo
2. Edad
3. Previsión o seguro de salud

# Bivariate and Multivariate Analysis

# Guiding Questions

# Visualizations

# Insights and questions

1. How the distribution of the data is affected if healthy newborns are not considered in the analysis?

In [40]:
df.columns

Index(['Sex', 'Age', 'Health insurance', 'Healthcare facility type',
       'Healthcare facility name', 'Length of stay', 'Discharge condition',
       'Primary diagnosis code', 'Primary diagnosis name', 'Year'],
      dtype='object')